# PATMOS-x v06r00 time series from monthly means

Monthly means for PATMOS-x v06r00 data have been computed with the script [PATMOS-x_monthlymean.py](../scripts/PATMOS-x_monthlymean.py). The monthly means are computed at the pixel level. That is, the monthly mean for a given variable is a 3D array of coordinates `(time, latitude, longitude)`. Here, `time` is a `datetime64[ns]` which indicates the month.

The monthly means have been computed for the following variables:
- `cloud_probability`
- `cloud_fraction`
- `tiwp`
- `tiwp_mixed`

Notes:
- Concerning `cloud_fraction`, based on https://doi.org/10.3390/rs8060511, which is not the official documentation (the [official documentation](https://www.ncei.noaa.gov/pub/data/sds/cdr/CDRs/AVHRR-HIRS_Cloud_Properties_PATMOS-x/AlgorithmDescription/AlgorithmDescription_ACHA_01B-1d.pdf) says very little) is obtained by:
  1. Centering an array of 3x3 pixels of the native satellite on the PATMOS-x pixel.
  2. Determining if the native pixel is cloudy based on `cloud_probability > 0.5` computed for the native pixel.
  3. Taking the average of the 3x3 boolean array.

  The `cloud_fraction` can be observed as a mean cloud probability ([source](https://doi.org/10.1175/JAMC-D-11-02.1)).

- `tiwp` is obtained by considering the cloud water path of those pixels which are indicated to have an ice cloud phase.

- `tiwp_mixed` is obtained by considering the cloud water path of those pixels which are indicated to have a mixed cloud phase.

- The cloud water paths are computed using the DCOMP (daytime-only) algorithm.

- PATMOS-x has a constant spatial grid.

- The monthly means files contain the variables `var_count`, e.g. `cloud_fraction_count`, which indicate how many values were used to compute the average for every pixel.

Based on the last point, I aim to take into account the number of data points available for each pixel when computing any average.

In [ ]:
# Load libraries
from pathlib import Path

import cartopy.crs as ccrs
import cmocean
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import xarray as xr

plt.style.use("../ccic.mplstyle")

In [ ]:
# Get monthly means file
ds = xr.open_dataset('/home/amell/src/ccic_climate_record_analysis/PATMOS-x_v06-monthlymean_198108-198109.nc')

In [ ]:
# Prepare the mask to use by resampling to the PATMOS-x coordinates
mask = xr.load_dataset('/scratch/ccic_record/data/mask_24.nc').mask

fig, axs = plt.subplots(ncols=2, subplot_kw=dict(projection=ccrs.PlateCarree()))

lons, lats = np.meshgrid(mask.longitude.data, mask.latitude.data)
m0 = axs[0].pcolormesh(lons, lats, mask, transform=ccrs.PlateCarree())
axs[0].coastlines()
axs[0].set_global()
axs[0].set_title('Mask before interpolation')
fig.colorbar(m0, ax=axs[0], shrink=0.2)


mask = mask.astype(int).interp(
    {
        'latitude': ds.latitude.data,
        'longitude': ds.longitude.data
    },
    method='nearest'
) == 1 # .astype(bool) doesn't work, since NaNs are converted to True


lons, lats = np.meshgrid(mask.longitude.data, mask.latitude.data)
m1 = axs[1].pcolormesh(lons, lats, mask, transform=ccrs.PlateCarree())
axs[1].coastlines()
axs[0].set_global()
axs[1].set_title('Mask after interpolation')
fig.colorbar(m1, ax=axs[1], shrink=0.2)

plt.show()

## Average maps of the different variables

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw=dict(projection=ccrs.PlateCarree()))

step = 10

var = 'cloud_probability'
m_cloud_probability = axs[0,0].contourf(
    ds.longitude.data[::step],
    ds.latitude.data[::step],
    # Below is to take into account the number of points available for each pixel
    ((ds[var] * ds[f"{var}_count"]).sum("time", skipna=True) / ds[f"{var}_count"].sum("time", skipna=True)).data[::step, ::step],
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=1,
    levels=11
)
m_cloud_probability.set_rasterized(True)
fig.colorbar(m_cloud_probability, ax=axs[0,0], shrink=0.5)
axs[0,0].set_title(rf'\verb+{var}+', loc='center')

var = 'cloud_fraction'
m_cloud_fraction = axs[0,1].contourf(
    ds.longitude.data[::step],
    ds.latitude.data[::step],
    ((ds[var] * ds[f"{var}_count"]).sum("time", skipna=True) / ds[f"{var}_count"].sum("time", skipna=True)).data[::step, ::step],
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=1,
    levels=11
)
m_cloud_fraction.set_rasterized(True)
fig.colorbar(m_cloud_fraction, ax=axs[0,1], shrink=0.5)
axs[0,1].set_title(rf'\verb+{var}+', loc='center')

norm = LogNorm(1e-2, 1e2, clip=True)
levels = np.logspace(-2, 2, 11)

var = 'tiwp'
m_tiwp = axs[1,0].contourf(
    ds.longitude.data[::step],
    ds.latitude.data[::step],
    ((ds[var] * ds[f"{var}_count"]).sum("time", skipna=True) / ds[f"{var}_count"].sum("time", skipna=True)).data[::step, ::step] * 1e-3, # Convert to kg/m2
    transform=ccrs.PlateCarree(),
    norm=norm,
    levels=levels
)
m_tiwp.set_rasterized(True)
fig.colorbar(m_tiwp, ax=axs[1,0], shrink=0.5, extend='both')
axs[1,0].set_title(rf'\verb+{var}+ [\si{{\kilo\gram\per\square\metre}}]', loc='center')

var = 'tiwp_mixed'
m_tiwp_mixed = axs[1,1].contourf(
    ds.longitude.data[::step],
    ds.latitude.data[::step],
    ((ds[var] * ds[f"{var}_count"]).sum("time", skipna=True) / ds[f"{var}_count"].sum("time", skipna=True)).data[::step, ::step] * 1e-3, # Convert to kg/m2
    transform=ccrs.PlateCarree(),
    norm=norm,
    levels=levels
)
m_tiwp_mixed.set_rasterized(True)
fig.colorbar(m_tiwp_mixed, ax=axs[1,1], shrink=0.5, extend='both')
axs[1,1].set_title(rf'\verb+{var}+ [\si{{\kilo\gram\per\square\metre}}]', loc='center')

for i, ax in enumerate(axs.ravel()):
    ax.coastlines()
    ax.set_global()
    ax.set_title(f'({chr(97 + i)})', loc='left')

fig.tight_layout()

plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw=dict(projection=ccrs.PlateCarree()))

step = 10

for i, (ax, var) in enumerate(zip(axs.ravel(), ['cloud_probability', 'cloud_fraction', 'tiwp', 'tiwp_mixed'])):
    var = f"{var}_count"
    c = ax.contourf(
        ds.longitude.data[::step],
        ds.latitude.data[::step],
        ds[var].sum(dim="time", skipna=True).data[::step, ::step], # Convert to kg/m2
        transform=ccrs.PlateCarree()
    )
    c.set_rasterized(True)
    fig.colorbar(c, ax=ax, shrink=0.5)
    ax.coastlines()
    ax.set_global()
    ax.set_title(f'({chr(97 + i)})', loc='left')
    ax.set_title(rf'\verb+{var}+', loc='center')

fig.tight_layout()

plt.show()

What is the numerical difference between `cloud_probability` and `cloud_fraction`?

In [ ]:
diff = ((ds['cloud_probability'] * ds["cloud_probability_count"]).sum("time", skipna=True) / ds["cloud_probability_count"].sum("time", skipna=True)).data - ((ds['cloud_fraction'] * ds["cloud_fraction_count"]).sum("time", skipna=True) / ds["cloud_fraction_count"].sum("time", skipna=True)).data

fig = plt.figure()
gs = fig.add_gridspec(nrows=2)

ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
c = ax1.contourf(ds.longitude.data[::10], ds.latitude.data[::10], diff[::10,::10], vmin=-1, vmax=1, cmap='coolwarm', transform=ccrs.PlateCarree())
c.set_rasterized(True)
ax1.set_title(r'\verb+cloud_probability+ $-$ \verb+cloud_fraction+', loc='center')
fig.colorbar(c, ax=ax1, shrink=0.75)
ax1.set_global()
ax1.coastlines()

ax2 = fig.add_subplot(gs[1])
ax2.hist(diff.flatten(), bins=np.linspace(-1, 1, 500, endpoint=True), density=True)
ax2.set_xlabel(r'\verb+cloud_probability+ $-$ \verb+cloud_fraction+')
ax2.set_ylabel('probability density function')

fig.tight_layout()

plt.show()

In [ ]:
weights_latitude = np.cos(np.deg2rad(ds.latitude))

In [ ]:
fig, (ax_cloud, ax_tiwp) = plt.subplots(nrows=2)

alpha = 0.3

for i, var in enumerate(['cloud_probability', 'cloud_fraction']):
    weights_count = ds[f'{var}_count'].astype(float)
    weights_count.data = 1 / np.where(weights_count == 0, np.nan, weights_count)
    weights = weights_latitude * weights_count
    weighted_var = ds[var].weighted(weights.fillna(0))
    weighted_var_masked = ds[var].where(mask == True).weighted(weights.fillna(0))

    color = f'C{i*2}'
    ax_cloud.plot(
        ds.time,
        weighted_var.mean(dim=("longitude", "latitude"), skipna=True).data,
        color=color,
        label=f'{var} average'
    )
    ax_cloud.fill_between(
        ds.time,
        weighted_var.quantile(0.16, dim=("longitude", "latitude"), skipna=True).data,
        weighted_var.quantile(0.84, dim=("longitude", "latitude"), skipna=True).data,
        alpha=alpha,
        color=color,
        label=f'{var} 16-84th percentiles'
    )

    color = f'C{i*2+1}'
    ax_cloud.plot(
        ds.time,
        weighted_var_masked.mean(dim=("longitude", "latitude"), skipna=True).data,
        color=color,
        label=f'{var} (masked) average'
    )
    ax_cloud.fill_between(
        ds.time,
        weighted_var_masked.quantile(0.16, dim=("longitude", "latitude"), skipna=True).data,
        weighted_var_masked.quantile(0.84, dim=("longitude", "latitude"), skipna=True).data,
        alpha=alpha,
        color=color,
        label=f'{var} (masked) 16-84th percentiles'
    )

ax_cloud.legend()
ax_cloud.set_ylim(0, 1)

for i, var in enumerate(['tiwp', 'tiwp_mixed']):
    weights_count = ds[f'{var}_count'].astype(float)
    weights_count.data = 1 / np.where(weights_count == 0, np.nan, weights_count)
    weights = weights_latitude * weights_count
    weighted_var = ds[var].weighted(weights.fillna(0))
    weighted_var_masked = ds[var].where(mask == True).weighted(weights.fillna(0))

    color = f'C{i*2}'
    ax_tiwp.plot(
        ds.time,
        weighted_var.mean(dim=("longitude", "latitude"), skipna=True).data * 1e-3,
        color=color,
        label=f'{var} average'
    )
    ax_tiwp.fill_between(
        ds.time,
        weighted_var.quantile(0.16, dim=("longitude", "latitude"), skipna=True).data * 1e-3,
        weighted_var.quantile(0.84, dim=("longitude", "latitude"), skipna=True).data  * 1e-3,
        alpha=alpha,
        color=color,
        label=f'{var} 16-84th percentiles'
    )

    color = f'C{i*2+1}'
    ax_tiwp.plot(
        ds.time,
        weighted_var_masked.mean(dim=("longitude", "latitude"), skipna=True).data * 1e-3,
        color=color,
        label=f'{var} (masked) average'
    )
    ax_tiwp.fill_between(
        ds.time,
        weighted_var_masked.quantile(0.16, dim=("longitude", "latitude"), skipna=True).data * 1e-3,
        weighted_var_masked.quantile(0.84, dim=("longitude", "latitude"), skipna=True).data * 1e-3,
        alpha=alpha,
        color=color,
        label=f'{var} (masked) 16-84th percentiles'
    )

ax_tiwp.legend()
ax_tiwp.set_ylim(1e-2, 1e2)
ax_tiwp.set_yscale('log')

plt.show()